# Lab 1 - Information Security

<ul>
  <li>Andrea Andreozzi (2163406)</li>
  <li>Riccardo Scalco (2155352)</li>
  <li>Sergio Cibecchini (2155353)</li>
  <li>Luca Ferrari (2166294)</li>
</ul>

# Variable environment
Fisrt of all we define some useful and we import numpy to use some useful functions with array

In [1]:
import numpy as np
import random

In [2]:
n = 5 # number of round
p = 11 # every number in this lab are in mod p
multiplicative_value_of_f = 2 # number that is multiply in the substitution function of AES
matrix_A = np.array([[2, 5], [1, 7]])
dim = 8 # length of the key and the dimension of the squared matrix
max_distance = 40 # max distance for brute force

We also define a method to read the pairs plaintext-ciphertext given in a file

In [3]:
# Function to read plaintext-ciphertext pairs from a file
def read_pairs_from_file(filepath):
    # Open the file at the given filepath in read mode
    with open(filepath, 'r') as f:
        # Read all lines from the file, stripping whitespace and ignoring empty lines
        lines = [line.strip() for line in f if line.strip()]

    # Initialize empty lists to store plaintexts and ciphertexts
    plaintexts, ciphertexts = [], []
    for line in lines:
        try:
            # Remove square brackets from the line
            line = line.replace('[', '').replace(']', '')
            # Split the line into two parts: plaintext (left) and ciphertext (right)
            left, right = line.split()  # Separate plaintext and ciphertext
            # Convert the plaintext (left) into a NumPy array of integers
            u = np.array(list(map(int, left.split(','))))
            # Convert the ciphertext (right) into a NumPy array of integers
            x = np.array(list(map(int, right.split(','))))
            # Append the plaintext and ciphertext to their respective lists
            plaintexts.append(u)
            ciphertexts.append(x)
        except Exception as e:
            # Print an error message if there is an issue parsing the line
            print(f"Error parsing the line: {line}")
            print(e)
    return plaintexts, ciphertexts

The following function is used to check if two array are equal. We will use it to check if plaintext (ciphertext) found with encryptor (decryptor) and the expected one are equal.

In [4]:
def test(result, target):
    print(result)
    if np.array_equal(result, target):
        print("Test passed")
    else:
        print("Test failed")

# Task 1 - Simplified AES-like cipher Encryption

This variables are used to test encryption and decryption of the first and second tasks

In [5]:
u = np.array([1, 0, 0, 0, 0, 0, 0, 0])
k = np.array([1, 0, 0, 0, 0, 0, 0, 0])

Then we define all functions that impement the linear AES system encryption

In [6]:
def subkey_generation(k):
    k1 = [k[0], k[2], k[4], k[6]]
    k2 = [k[0], k[1], k[2], k[3]]
    k3 = [k[0], k[3], k[4], k[7]]
    k4 = [k[0], k[3], k[5], k[6]]
    k5 = [k[0], k[2], k[5], k[7]]
    k6 = [k[2], k[3], k[4], k[5]]
    return np.array([
        k1, k2, k3, k4, k5, k6
    ])

In [7]:
def subkey_sum(w, ki):
    subkey = np.tile(ki, 2)  # k = ki + ki
    return (w + subkey[:len(w)]) % p

In [8]:
def substitution(v):
    # f = 2*v %p
    return (multiplicative_value_of_f * v) % p

In [9]:
def transposition(y):
    # flipped the second half of vector yi
    return np.array(y)[[0, 1, 2, 3, 7, 6, 5, 4]]

In [10]:
def linear(z):
    # linear transformation: write (by rows) vector z to 2 × 4 matrix Z
    Z = z.reshape(2, 4)
    w = (matrix_A @ Z) % p
    return w.flatten()

In [11]:
def encryption(u, k):
    subkey = subkey_generation(k)
    w = u.copy()
    for i in range(n):
        v = subkey_sum(w, subkey[i])
        y = substitution(v)
        z = transposition(y)
        if i != n-1:
            w = linear(z)
    x = subkey_sum(z, subkey[n])
    return x

Finally we can check if the encryption method defined above is correct.

In [12]:
x_test = [4, 0, 0, 9, 7, 0, 0, 3] # the expected result
x = encryption(u, k)
test(x, x_test)

[4 0 0 9 7 0 0 3]
Test passed


# Task 2 - Simplified AES-like cipher: Decryption

In [13]:
def inverse_subkey_sum(w, ki):
    k = np.tile(ki, 2)  # k = ki + ki
    return (w - k[:len(w)] + p) % p # I add p to avoid negative values

In [14]:
def inverse_substitution(v):
    inverse_of_multiplicative_value_of_f = pow(multiplicative_value_of_f, -1, p)
    return (v * inverse_of_multiplicative_value_of_f) % p

In [15]:
def inverse_transposition(y):
    return np.array(y)[[0, 1, 2, 3, 7, 6, 5, 4]]

In [16]:
def modular_inverse_matrix(A):
    det = int(np.round(np.linalg.det(A)))  # Determinante modulo p
    det_inv = pow(det, -1, p)  # Inverso del determinante mod p
    A_inv = np.round(det_inv * np.linalg.inv(A) * det)  # Matrice inversa mod p
    return A_inv.astype(int) % p # Converti in interi

In [17]:
def inverse_linear(z):
    A_inv = modular_inverse_matrix(matrix_A)
    w = z.reshape(2, 4)
    w = (A_inv @ w) % p
    return w.flatten()

In [18]:
def decryption(x, k):
    subkey = subkey_generation(k)
    w = inverse_subkey_sum(x, subkey[n])  # Annulla l'ultima somma
    for i in range(n - 1, -1, -1):  # Decifra nei round inversi
        z = inverse_linear(w) if i != n - 1 else w
        y = inverse_transposition(z)
        v = inverse_substitution(y)
        w = inverse_subkey_sum(v, subkey[i])
    return w.copy()

Then, we are able to check the encryptor function. The value of `x` is one founded in previous task.

In [19]:
u = decryption(x, k)
u_test = [1, 0, 0, 0, 0, 0, 0, 0]
test(u, u_test)

[1 0 0 0 0 0 0 0]
Test passed


# Task 3 - Identify the cipher vulnerability

In [20]:
def generate_matrix_A_B():
    # Initialize matrices A and B with zeros
    A_matrix = np.zeros((dim, dim), dtype=int)
    B_matrix = np.zeros((dim, dim), dtype=int)

    # Compute matrix A: E(e_j, 0)
    for j in range(dim):
        # Create the unit vector e_j
        e_j = np.zeros(dim, dtype=int)
        e_j[j] = 1
        # Use e_j as the key and a zero vector as the input
        k = e_j
        u = np.zeros(dim, dtype=int)
        # Encrypt and store the result in the j-th column of A
        A_matrix[:, j] = encryption(u, k)

    # Compute matrix B: E(0, e_j)
    for j in range(dim):
        # Create the unit vector e_j
        e_j = np.zeros(dim, dtype=int)
        e_j[j] = 1
        # Use a zero vector as the key and e_j as the input
        k = np.zeros(dim, dtype=int)
        u = e_j
        # Encrypt and store the result in the j-th column of B
        B_matrix[:, j] = encryption(u, k)

    # Return the computed matrices A and B
    return A_matrix, B_matrix

In [21]:
def validate_matrixs(A, B, u, k):
    # x = E(k, u) = Ak + Bu mod p
    x_true = encryption(u, k)
    x_calculated = (A @ k + B @ u) % p

    # Check if the encryption property holds
    if np.array_equal(x_true, x_calculated):
        print("Test passed")
    else:
        print("Test failed")

In [22]:
A, B = generate_matrix_A_B()
print("Matrix A:")
print(A)
print("\nMatrix B:")
print(B)
validate_matrixs(A, B, u, k)

Matrix A:
[[ 9  0  1  6  0  0  1 10]
 [ 0  8  6  2  2  9  0  0]
 [ 0  6  0  8  3 10  0  0]
 [ 6  0  0  8  0  1  6  6]
 [ 2  0  1 10  0  0  1  3]
 [ 0  1  8  4  9  6  0  0]
 [ 0 10  0  5  7  6  0  0]
 [ 3  0  0  1  0  1  4  8]]

Matrix B:
[[6 0 0 3 3 0 0 0]
 [0 6 3 0 0 3 0 0]
 [0 3 6 0 0 0 3 0]
 [3 0 0 6 0 0 0 3]
 [5 0 0 0 4 0 0 8]
 [0 5 0 0 0 4 8 0]
 [0 0 5 0 0 8 4 0]
 [0 0 0 5 8 0 0 4]]
Test passed


# Task 4 - Linear cryptanalysis KPA

In [23]:
def recover_key(u, x, A, B):
    # Compute the modular inverse of matrix A modulo p
    A_inv = modular_inverse_matrix(A)
    # Compute the product of matrix B and vector u modulo p
    Bu = (B @ u)
    # Compute the difference between the ciphertext x and Bu modulo p
    diff = (x - Bu)
    # Compute the key k using the formula: k = A_inv @ diff % p
    k = (A_inv @ diff) % p
    return k

In [24]:
def validateKey(plaintexts, ciphertexts, k):
    # Iterate over all plaintext-ciphertext pairs
    for i in range(len(plaintexts)):
        u = plaintexts[i]
        x = ciphertexts[i]
        # Compute the encryption of the plaintext u using the key k
        x_test = encryption(u, k)
        # Check if the computed ciphertext matches the given ciphertext
        if not np.array_equal(x, x_test):
            print("Test failed at index ", i)
            print("expected:" , x , "actual:" , x_test)
            return
    print("Test passed")

In [25]:
# Generate the matrices A and B using the function from task3
A, B = generate_matrix_A_B()

# Define the filepath to the file containing plaintext-ciphertext pairs
filepath = "KPApairsH_linear.txt"
# Read the plaintext-ciphertext pairs from the file
plaintexts, ciphertexts = read_pairs_from_file(filepath)

k = recover_key(plaintexts[0], ciphertexts[0], A, B)
print(k)

validateKey(plaintexts, ciphertexts, k)

[10  4 10  2  6  7  8  2]
Test passed


#Task 5 - “Nearly linear” simplified AES-like cipher

In [26]:
substitution_map = {0: 0, 1: 2, 2: 4, 3: 8, 4: 6, 5: 10, 6: 1, 7: 3, 8: 5, 9: 7, 10: 9}

In [27]:
def substitution_nearly_linear(v):
    # substitution: apply substitution map to vector v
    for i in range(len(v)):
        v[i] = substitution_map[v[i]]
    return v

In [28]:
def encryption_nearly_linear(u, k):
    subkey = subkey_generation(k)
    w = u.copy()
    for i in range(n):
        v = subkey_sum(w, subkey[i])
        y = substitution_nearly_linear(v)
        z = transposition(y)
        if i != n-1:
            w = linear(z)
    x = subkey_sum(z, subkey[n])
    return x

In [29]:
k = [1, 0, 0, 0, 0, 0, 0, 0]
u = [1, 0, 0, 0, 0, 0, 0, 0]
x_test = [9, 0, 0, 0, 5, 0, 0, 6] # the expected result
x = encryption_nearly_linear(u, k)
test(x, x_test)

[9 0 0 0 5 0 0 6]
Test passed


# Task 6 - Linear cryptanalysis of a “nearly linear” cipher

In [30]:
def near_keys(start_key, distance):
    key_len = len(start_key)  # Calcolo dinamico della lunghezza

    def backtrack(pos, current, remaining):
        if pos == key_len:
            if remaining == 0:
                yield (start_key + np.array(current)) % p
            return
        for delta in range(-remaining, remaining + 1):
            current[pos] = delta
            yield from backtrack(pos + 1, current, remaining - abs(delta))

    current = [0] * key_len
    yield from backtrack(0, current, distance)

In [31]:
def key_test(k, plaintexts, ciphertexts):
    if not np.array_equal(encryption_nearly_linear(plaintexts[0], k), ciphertexts[0]):
        return False
    for u, x in zip(plaintexts, ciphertexts):
        if not np.array_equal(encryption_nearly_linear(u, k), x):
            return False
    return True

In [ ]:
def brute_force(start_key, plaintexts, ciphertexts, max_distance):
    for distance in range(1, max_distance + 1):
        count = 0
        print(f"Testing keys at distance {distance}")
        for key in near_keys(start_key, distance):
            count += 1
            if key_test(key, plaintexts, ciphertexts):
                return key
        print(count, "tested keys at distance", distance)
    return None

In [33]:
def recover_one_key(u, x, A, B, C):
    # Compute the modular inverse of matrix A
    A_inv = modular_inverse_matrix(A)
    # Compute the product of matrix B and vector u
    Bu = (B @ u)
    # Compute the product of matrix C and vector x
    Cx = (C @ x)
    # Compute the difference between the Cx and Bu
    diff = (-Cx - Bu)
    # Compute the key k using the formula: k = A_inv @ diff % p
    k = (A_inv @ diff) % p
    return k

In [34]:
def key_approximation(A, B, C, plaintexts, ciphertexts):
    #trovo la chiave del primo plaintext-ciphertext
    k = []
    for i in range(len(plaintexts)):
        u = plaintexts[i]
        x = ciphertexts[i]
        k.append(recover_one_key(u, x, A, B, C))

    num_keys = len(k)  # Numero di vettori in k
    key_length = len(k[0])  # Lunghezza di ciascun vettore (8 in questo caso)

    # Creiamo una lista per le medie
    averages = []
    # Calcoliamo la media per ogni indice j (0, 1, ..., 7)
    for j in range(key_length):
        avg = sum(k[i][j] for i in range(num_keys)) / num_keys
        averages.append(round(avg))
    return averages

In [35]:
filepath = "KPApairsH_nearly_linear.txt"
plaintexts, ciphertexts = read_pairs_from_file(filepath)
A, B = generate_matrix_A_B()
C = np.identity(dim)
key = key_approximation(A, B, C, plaintexts, ciphertexts)
final_key = brute_force(key, plaintexts, ciphertexts, max_distance)
if final_key is not None:
    print("Found key", final_key)
else:
    print("No valid key found")


Testing keys at distance 1
16 tested keys at distance 1
Testing keys at distance 2
128 tested keys at distance 2
Testing keys at distance 3
688 tested keys at distance 3
Testing keys at distance 4
2816 tested keys at distance 4
Testing keys at distance 5
9424 tested keys at distance 5
Testing keys at distance 6
27008 tested keys at distance 6
Testing keys at distance 7
68464 tested keys at distance 7
Testing keys at distance 8
157184 tested keys at distance 8
Testing keys at distance 9
332688 tested keys at distance 9
Testing keys at distance 10
658048 tested keys at distance 10
Testing keys at distance 11
1229360 tested keys at distance 11
Testing keys at distance 12
2187520 tested keys at distance 12
Testing keys at distance 13
Found key: [ 5 10  4  9  2  3  3  8]
Found key [ 5 10  4  9  2  3  3  8]


# Task 7 - Non linear simplified AES-like cipher

In [36]:
u = np.array([1, 0, 0, 0, 0, 0, 0, 0])
k = np.array([1, 0, 0, 0])

In [37]:
def subkey_generation_non_linear(k):
    k1 = [k[0], k[1], k[2], k[3]]
    k2 = [k[0], k[1], k[3], k[2]]
    k3 = [k[1], k[2], k[3], k[0]]
    k4 = [k[0], k[3], k[1], k[2]]
    k5 = [k[2], k[3], k[0], k[1]]
    k6 = [k[1], k[3], k[0], k[2]]
    return np.array([
        k1, k2, k3, k4, k5, k6
    ])

In [38]:
def modular_inverse(v):
    """Calcola l'inverso moltiplicativo modulo p per ogni elemento di v"""
    inverses = []
    for x in v:
        #nel nostro caso p = 11, l'unico numero non invertibile è 0
        if np.gcd(x, p) == 1:
            inverses.append(pow(int(x), -1, p))
        else:
            inverses.append(0)
    return np.array(inverses)

In [39]:
def substitution_non_linear(v):
    v_inv = modular_inverse(v)  # Calcola l'inverso modulo p
    return (multiplicative_value_of_f * v_inv) % p  # Applica la substitution

In [40]:
def encryption_non_linear(u, k):
    subkey = subkey_generation_non_linear(k)
    w = u.copy()
    for i in range(n):
        v = subkey_sum(w, subkey[i])
        y = substitution_non_linear(v)
        z = transposition(y)
        if i != n-1:
            w = linear(z)
    x = subkey_sum(z, subkey[n])
    return x

We also define the functions to decrypt (not required but useful for next task).

In [41]:
def inverse_substitution_non_linear(y):
    y_inv = (y * pow(multiplicative_value_of_f, -1, p)) % p  # Reverse substitution step
    return modular_inverse(y_inv)  # Apply modular inverse

In [42]:
def decryption_non_linear(x, k):
    subkey = subkey_generation_non_linear(k)
    w = inverse_subkey_sum(x, subkey[n])  # Undo the last subkey sum
    for i in range(n - 1, -1, -1):  # Reverse the encryption rounds
        z = inverse_linear(w) if i != n - 1 else w
        y = inverse_transposition(z)
        v = inverse_substitution_non_linear(y)
        w = inverse_subkey_sum(v, subkey[i])
    return w.copy()

In [43]:
x_test = [5, 0, 3, 2, 5, 2, 1, 1]
x = encryption_non_linear(u, k)
test(x, x_test)
u = decryption_non_linear(x, k)
test(u, u_test)

[5 0 3 2 5 2 1 1]
Test passed
[1 0 0 0 0 0 0 0]
Test passed


# Task 8 - Meet in the middle attack

In [44]:
dim = 4
n_guesses = 50000

In [45]:
def generate_random_key(dim, p):
    return np.array([random.randint(0, p-1) for _ in range(dim)])

In [46]:
def generate_key_guesses(n_guesses):
    key_prime = [generate_random_key(dim, p) for _ in range(n_guesses)]
    key_second = [generate_random_key(dim, p) for _ in range(n_guesses)]
    return key_prime, key_second

In [47]:
def generate_plaintext_ciphertext_from_keys(u, x, keys_prime, keys_second):
    ciphertexts = {}
    plaintexts = {}
    for kp in keys_prime:
        ciphertexts[tuple(kp)] = encryption(u, kp)
    for ks in keys_second:
        plaintexts[tuple(ks)] = decryption(x, ks)
    return ciphertexts, plaintexts

In [48]:
def look_for_matches(generated_ciphertexts, generated_plaintexts):
    matching_pairs = []
    for ks in generated_ciphertexts.keys():
        for kp in generated_plaintexts.keys():
            if np.array_equal(generated_ciphertexts[ks], generated_plaintexts[kp]):
                matching_pairs.append([{ks: generated_ciphertexts[ks]}, {kp: generated_plaintexts[kp]}])
    return matching_pairs

In [49]:
# Define the filepath to the file containing plaintext-ciphertext pairs
filepath = "KPApairsH_non_linear.txt"

# Read the plaintext-ciphertext pairs from the file
original_plaintexts, original_ciphertexts = read_pairs_from_file(filepath)

# Generate random key guesses
keys_prime, keys_second = generate_key_guesses(n_guesses)

# Generate encryption & decryption mappings
generated_ciphertexts, generated_plaintexts = generate_plaintext_ciphertext_from_keys(
    original_plaintexts[0],
    original_ciphertexts[0],
    keys_prime,
    keys_second
)

# Look for key matches
results = look_for_matches(generated_ciphertexts, generated_plaintexts)

print(f"Found {len(results)} matching key pairs:")
for y, match in enumerate(results):
    print(match)

    k1 = match[y][0]
    k2 = match[y][1]
    for i in range(len(original_plaintexts)):
        u = original_plaintexts[i]
        x = original_ciphertexts[i]
        # Encrypt the plaintext with the first key
        encrypted_with_k1 = encryption(u, k1)
        # Decrypt the ciphertext with the second key
        encrypted_with_k2 = encryption(encrypted_with_k1, k2)
        if np.array_equal(encrypted_with_k2, x):
            print(f"The key number {y + 1}  works for encrypt world number {i+1} of the given pairs")
        else:
            print(f"The key number {y + 1}  doesn't works for encrypt world number {i+1} of the given pairs")
        decrypted_with_k2 = decryption(x, k2)
        decrypted_with_k1 = decryption(decrypted_with_k2, k1)
        # Print the results
        if np.array_equal(decrypted_with_k1, u):
            print(f"The key number {y + 1}  works for decrypt world number {i+1} of the given pairs")
        else:
            print(f"The key number {y + 1} doesn't works for decrypt world number {i+1} of the given pairs")

IndexError: index 4 is out of bounds for axis 0 with size 4